<a href="https://colab.research.google.com/github/Nikhitha-S-Pavan/Machine-Learning-Algorithms/blob/main/email_spam_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#import all the needed libraries
import mailbox
%matplotlib inline
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas
import sklearn
#import cPickle
import numpy as np

#function to get email text from email body
def getmailtext(message): #getting plain text 'email body'
    body = None
    #check if mbox email message has multiple parts
    if message.is_multipart():
        for part in message.walk():
            if part.is_multipart():
                for subpart in part.walk():
                    if subpart.get_content_type() == 'text/plain':
                        
                        body = subpart.get_payload(decode=True)
            elif part.get_content_type() == 'text/plain':
                body = part.get_payload(decode=True)
    #if message only has a single part            
    elif message.get_content_type() == 'text/plain':
        body = message.get_payload(decode=True)
    #return mail text which concatenates both mail subject and body
    mailtext=str(message['subject'])+" "+str(body)
    print(mailtext)
    return mailtext


#read spam mbox email file
mbox = mailbox.mbox('/content/Starred.mbox')
smbox = mailbox.mbox('/content/Spam.mbox')

star = []
count =0
#create list which contains mail text for each spam email message
for message in mbox:    
    star.append(getmailtext(message))
    if count>601:
        break
    count+=1
    #break
count =0
spam = []
#create list which contains mail text for each spam email message
for message in smbox:    
    spam.append(getmailtext(message))
    if count>601:
        break
    count+=1

In [ ]:

#create 2 dataframes for ham spam mails which contain the following info-
#Mail text, mail length, mail is ham/spam label
import pandas as pd
spam_df = pd.DataFrame(spam, columns=["message"])
spam_df["label"] = "spam"

spam_df['length'] = spam_df['message'].map(lambda text: len(text))
print(spam_df.head())

ham_df = pd.DataFrame(star, columns=["message"])
ham_df["label"] = "ham"

ham_df['length'] = ham_df['message'].map(lambda text: len(text))
print(ham_df.head())

In [ ]:

#merge and shuffle dataframes for ham/spam mails
mail_df = pd.concat([spam_df,ham_df])
from sklearn.utils import shuffle
mail_df = shuffle(mail_df)
len(mail_df)

In [ ]:
from sklearn.model_selection import train_test_split
mail_train, mail_test, y_train, y_test = train_test_split(mail_df['message'],mail_df['label'],test_size=0.33, random_state=42)

In [ ]:
#for each word in the email text, get the base form of the word and return the list of base words
def split_into_lemmas(message):
    message = message.lower()
    words = TextBlob(message).words
    # for each word, take its "base form" = lemma
    print([word.lemma for word in words])
    return [word.lemma for word in words]

In [ ]:
#function to apply the count vectorizer(BOW) and TF-IDF transforms to a set of input features
def features_transform(mail):
    #get the bag of words for the mail text
    bow_transformer = CountVectorizer(analyzer=split_into_lemmas).fit(mail_train)
    print(bow_transformer)
    #print(len(bow_transformer.vocabulary_))
    messages_bow = bow_transformer.transform(mail)
    #print sparsity value
    print('sparse matrix shape:', messages_bow.shape)
    print('number of non-zeros:', messages_bow.nnz) 
    print('sparsity: %.2f%%' % (100.0 * messages_bow.nnz / (messages_bow.shape[0] * messages_bow.shape[1])))
    #apply the TF-IDF transform to the output of BOW
    tfidf_transformer = TfidfTransformer().fit(messages_bow)
    messages_tfidf = tfidf_transformer.transform(messages_bow)
    
    #return result of transforms
    return messages_tfidf

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
#transform training set features into a set of useful features to build models
train_features=features_transform(mail_train)